In [6]:
import numpy as np
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split


class MTO:
    def __init__(self, objective_function, bounds, population_size=1, max_iter=2, alpha=0.2, beta=2.0):
        print("intialization")
        self.objective_function = objective_function
        self.bounds = bounds
        self.population_size = population_size
        self.max_iter = max_iter
        self.alpha = alpha
        self.beta = beta

    def run(self):
        print("In run")
        dim = len(self.bounds)
        lb = [self.bounds[i][0] for i in range(dim)]
        ub = [self.bounds[i][1] for i in range(dim)]
        pop = self.initialize_population(dim)
        print("population for this run:",pop)
        fit = [self.objective_function(pop[i]) for i in range(self.population_size)]
        print("objetive function applied to all the population , accuracy results are as follows",fit)
        best_index = np.argmin(fit)
        print("best_index",best_index)
        best_solution = pop[best_index]
        print("best_soultion",best_solution)
        for i in range(self.max_iter):
            for j in range(self.population_size):
                print("iteration number",i)
                if j == best_index:
                    continue
                x = pop[j]
                v = self.update_mother_tree(pop, j, best_index)
                print("v value after update mother tree",v)
                u = np.random.uniform(size=dim)
                a = 2.0 * self.alpha * u - self.alpha
                r1 = np.random.uniform(size=dim)
                r2 = np.random.uniform(size=dim)
                y = v + a * np.abs(r1 * best_solution - x) - a * np.abs(r2 * x - v)
                y = np.maximum(np.minimum(y, ub), lb)
                print("values",x,v,u,a,r1,r2,y)
                f_y = self.objective_function(y)
               
                if f_y < fit[j]:
                    pop[j] = y
                    fit[j] = f_y
                    if f_y < fit[best_index]:
                        best_index = j
                        best_solution = y
        print("best solution in this population is",best_solution)
        return best_solution

    def initialize_population(self, dim):
        print("intialize population")
        return np.random.uniform(size=(self.population_size, dim))

    def update_mother_tree(self, pop, j, best_index):
        print("update mother tree")
        dim = len(self.bounds)
        x = pop[j]
        best_solution = pop[best_index]
        r = np.random.uniform(size=dim)
        print("r value",r)
        v = (1.0 - r) * best_solution + r * np.mean(pop, axis=0)
        print("v value",v)
        beta = self.beta * (1.0 - float(j+1) / self.population_size)
        print("beta",beta)
        u = np.random.uniform(size=dim)
        print("u value",u)
        a = 2.0 * beta * u - beta
        print("a value",a)
        y = v + a * np.abs(best_solution - x)
        print("y value",y)
        return np.maximum(np.minimum(y, 1.0), 0.0)





# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess the data
x_train = x_train.reshape(-1, 784).astype('float32') / 255.0
x_test = x_test.reshape(-1, 784).astype('float32') / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Define the model architecture
def create_model(n_layers, n_units, dropout_rate):
    print("creating model")
    model = Sequential()
    for i in range(n_layers):
        if i == 0:
            model.add(Dense(n_units, activation='relu', input_shape=(784,)))
        else:
            model.add(Dense(n_units, activation='relu'))
        model.add(Dropout(dropout_rate))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Define the objective function to be optimized
def objective_function(x):
    print("runing objective function for population",x)
    n_layers = int(np.round(x[0]))
    n_units = int(np.round(x[1]))
    dropout_rate = x[2]
    model = create_model(n_layers, n_units, dropout_rate)
    model.fit(x_train, y_train, batch_size=128, epochs=10, verbose=0)
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    print("for population x=",x,"we have loss",loss,"and accuracy of", accuracy)
    return -accuracy  # We maximize accuracy, so we need to minimize the negative of accuracy

# Set the bounds for the variables to be optimized
bounds = [(1, 5), (32, 256), (0, 0.5)]

# Create an instance of the MTO algorithm
mto = MTO(objective_function, bounds, population_size=2, max_iter=2)

print("calling mto")
# Run the MTO algorithm
best_solution = mto.run()


# Print the best solution found
print('Best solution found: {}'.format(best_solution))


intialization
calling mto
In run
intialize population
population for this run: [[0.61966771 0.57297186 0.98538065]
 [0.35734271 0.36521654 0.37237464]]
runing objective function for population [0.61966771 0.57297186 0.98538065]
creating model
for population x= [0.61966771 0.57297186 0.98538065] we have loss 2.301025867462158 and accuracy of 0.11349999904632568
runing objective function for population [0.35734271 0.36521654 0.37237464]
creating model
for population x= [0.35734271 0.36521654 0.37237464] we have loss 0.2649429738521576 and accuracy of 0.9261999726295471
objetive function applied to all the population , accuracy results are as follows [-0.11349999904632568, -0.9261999726295471]
best_index 1
best_soultion [0.35734271 0.36521654 0.37237464]
iteration number 0
update mother tree
r value [0.67477392 0.81085426 0.19446006]
v value [0.44584775 0.44944618 0.43197723]
beta 1.0
u value [0.97876798 0.21712082 0.39968995]
a value [ 0.95753596 -0.56575837 -0.20062009]
y value [0.69703